In [2]:
# -*- coding: utf-8 -*-
"""
File: wsh_webquery_DB_download.py 
Parses HTTPResponse objects returned from webservice query

Created on Tues Sep  7 11:49:53 2021
Last modified: 5/5/22

@author: clj585
"""

###################################################
# Loads packages 
###################################################

import urllib.request
import xmltodict
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time

# Define path here 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/"

###################################################
# Send Webservice Query & pull data 
###################################################

"""
Function: formURL

Returns URL generated for WSH Enchilada webservice query 

@params: 
   stock_sym_list: list of strings of stock symbols
   class: string or ideally only 1 string
   from_dt: start date formatted as string
   to_dt: end date of query as string
"""
def formURL(stock_sym_list, evtclass, from_dt, to_dt):
    
    home = "https://enchilada.wallstreethorizon.com/webservice6.asp"
    cust_id = "?c=NORTHWESTERN"
    passw = "&p=4IQD42P0"
    
    stock_syms = "&stock_symbols="+",".join(stock_sym_list)
    classes = "&classes="+evtclass
    
    fromdt = "&from=" + from_dt
    todt = "&to=" + to_dt
    
    v = "&v=2"
    opts = "&o=EVENTS"
    
    url_str = (home + cust_id + passw + 
               stock_syms + 
               classes + fromdt + todt + 
               v + opts)
    return url_str

"""
Function: perDelta

Returns an iterable with all the starting dates of time intervals.
Breaks the history time window up into 9-day windows. 

@params: 
   start: string formatted date
   end:also string formatted end date
   delta: time-skip needs to be in days
"""
def perDelta(start, end, delta):
    start = datetime.strptime(start, "%m/%d/%y")
    end = datetime.strptime(end, "%m/%d/%y")
    curr = start
    while curr < end:
        yield curr
        curr += delta

###################################################
# Process columns of the DateBreaks files and save
###################################################

# PARAMETERS AND LISTS CAN BE MODIFIED HERE # 
stock_sym_list = ["*"]
# Entire list of desired event class types
#class_list = ["BYBK", "CC", "DB", 
#              "DIV", "DIVSR", "ED", "EPS", 
#              "FDIV", "FQ", "OPTION", 
#              "QE", "SEC"]
class_list = ['DB']

# ToDo Will: UPDATE BASED ON LOGS 
hist_from_dt = (datetime.today()-relativedelta(years=1)).strftime(
    "%m/%d/%y")
hist_to_dt = (datetime.today()).strftime("%m/%d/%y")
    
for evt in class_list:          # if stock wildcard only 1 evt at a time

    # Sleep condition avoid timeout
    if(class_list.index(evt)%4 == 0 and class_list.index(evt)!=0): 
        time.sleep(30)    
    
    # Generate 1-day intervals for query window
    for start_dt in perDelta(hist_from_dt, 
                             hist_to_dt,
                             timedelta(days=1)): 
        
        # Formatting as strings 
        from_dt = start_dt.strftime("%m/%d/%y")
        to_dt = (start_dt + relativedelta(days=1)).strftime(
            "%m/%d/%y")
        
        # Formulate URL for webservice query here 
        url3 = formURL(stock_sym_list, evt, 
                       from_dt, to_dt)
        print(url3)

        # Send, process request and save tree structure 
        rsp = urllib.request.urlopen(url3)
        string = rsp.read().decode('utf-8')
    
        # Conversion to dictionary 
        dictdata = xmltodict.parse(string)

        # For each dictionary list (keyed by event class) get data 
        for key in dictdata["webservice6"].keys():
            dataf = pd.DataFrame()                  # placeholder
            
            if key != "error":
                # Check if OrderedDict or list
                if(type(dictdata["webservice6"][key]) != list):
                    print("Are you here?")
                    dct = dictdata["webservice6"][key]
                
                    # Save to data frame and reformat
                    d1 = pd.DataFrame(dct.items(), columns=None)
                    d1 = d1.T
                    d1.rename(columns=d1.iloc[0], inplace = True)
                    d1.drop(d1.index[0], inplace = True)
                    dataf = pd.concat([dataf, d1])
                
                else:
                    print("It will be a list!")
                    for i in range(len(dictdata["webservice6"][key])):
                
                        # Number of dictionaries equal to number of event id's 
                        dct = dictdata["webservice6"][key][i]
                
                        # Save to data frame and reformat
                        d1 = pd.DataFrame(dct.items(), columns=None)
                        d1 = d1.T
                        d1.rename(columns=d1.iloc[0], inplace = True)
                        d1.drop(d1.index[0], inplace = True)
                        
                        dataf = pd.concat([dataf, d1])
                    
                # Process the data frame here 
                print(dataf.columns)
                dataf[['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
                      'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
                      'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
                      'created', 'updated', 'return_time']] = dataf[['event_id', 'company_id', 'stock_symbol', 
                                                             'isin', 'company_name',
                                                             'quarter', 'fiscal_year', 'echangetype',
                                                             'prior_earnings_date',
                                                             'earnings_date', 'time_of_day', 
                                                             'earnings_date_status', 'audit_source',
                                                             'created', 'updated', 
                                                             'return_time']].astype("string")
                # Datetime columns manipulate 
                dataf['fiscal_year'] = dataf['fiscal_year'].astype(int)
                dataf['prior_earnings_date'] = pd.to_datetime(dataf['prior_earnings_date'])
                dataf['earnings_date'] = pd.to_datetime(dataf['earnings_date'])
                dataf['created'] = pd.to_datetime(dataf['created'])
                dataf['updated'] = pd.to_datetime(dataf['updated'])
                dataf['return_time'] = pd.to_datetime(dataf['return_time'])
                
                if 'option_expiration_date' in dataf.columns:
                    dataf[["option_expiration_code", "option_expiration_date"]] = dataf[[
                        "option_expiration_code", "option_expiration_date"]].astype("string")
                    dataf['option_expiration_date'] = pd.to_datetime(dataf['option_expiration_date'])
        
                # Create asofdate (date) column
                dataf['date'] = start_dt
                dataf['date'] = pd.to_datetime(dataf['date'])
        
                if 'prelim_earnings_date' in dataf.columns:
                    dataf['prelim_earnings_date'] = dataf['prelim_earnings_date'].astype("string")
                    dataf['prelim_earnings_date'] = pd.to_datetime(dataf['prelim_earnings_date'])
        
                dt_lbl = start_dt.strftime("%m-%d-%y")
                
                # Save to desired path (ToDo: Change path as needed!)
                print(dataf.columns)
                print(len(dataf))
                print(onedrive_pth + f"temp_{key}/{key}_webserv{dt_lbl}.parquet")
                dataf.to_parquet(onedrive_pth + f"temp_{key}/{key}_webserv{dt_lbl}.parquet")

            else:
                print("no data in this date range for ", evt, ", ",
                      from_dt, " to ", to_dt, "! :)")
        

https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=10/03/21&to=10/04/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'prelim_earnings_date',
       'option_expiration_code', 'option_expiration_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'prelim_earnings_date',
       'option_expiration_code', 'option_expiration_date', 'date'],
      dtype='object')
223
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/tem

Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date', 'date'],
      dtype='object')
494
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_db/db_webserv10-11-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=10/12/21&to=10/13/21&

Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date', 'date'],
      dtype='object')
619
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_db/db_webserv10-19-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=10/20/21&to=10/21/21&

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date', 'date'],
      dtype='object')
1813
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_db/db_webserv10-28-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=1

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date', 'date'],
      dtype='object')
1540
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_db/db_webserv11-05-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=1

...

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date'],
      dtype='object')
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'quarter', 'fiscal_year', 'echangetype', 'prior_earnings_date',
       'earnings_date', 'time_of_day', 'earnings_date_status', 'audit_source',
       'created', 'updated', 'return_time', 'option_expiration_code',
       'option_expiration_date', 'prelim_earnings_date', 'date'],
      dtype='object')
495
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_db/db_webserv05-25-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=05

no data in this date range for  DB ,  06/07/22  to  06/08/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=06/08/22&to=06/09/22&v=2&o=EVENTS
no data in this date range for  DB ,  06/08/22  to  06/09/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=06/09/22&to=06/10/22&v=2&o=EVENTS
no data in this date range for  DB ,  06/09/22  to  06/10/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=06/10/22&to=06/11/22&v=2&o=EVENTS
no data in this date range for  DB ,  06/10/22  to  06/11/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=06/11/22&to=06/12/22&v=2&o=EVENTS
no data in this date range for  DB ,  06/11/22  to  06/12/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD4

no data in this date range for  DB ,  07/16/22  to  07/17/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=07/17/22&to=07/18/22&v=2&o=EVENTS
no data in this date range for  DB ,  07/17/22  to  07/18/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=07/18/22&to=07/19/22&v=2&o=EVENTS
no data in this date range for  DB ,  07/18/22  to  07/19/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=07/19/22&to=07/20/22&v=2&o=EVENTS
no data in this date range for  DB ,  07/19/22  to  07/20/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=07/20/22&to=07/21/22&v=2&o=EVENTS
no data in this date range for  DB ,  07/20/22  to  07/21/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD4

no data in this date range for  DB ,  08/24/22  to  08/25/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=08/25/22&to=08/26/22&v=2&o=EVENTS
no data in this date range for  DB ,  08/25/22  to  08/26/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=08/26/22&to=08/27/22&v=2&o=EVENTS
no data in this date range for  DB ,  08/26/22  to  08/27/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=08/27/22&to=08/28/22&v=2&o=EVENTS
no data in this date range for  DB ,  08/27/22  to  08/28/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=DB&from=08/28/22&to=08/29/22&v=2&o=EVENTS
no data in this date range for  DB ,  08/28/22  to  08/29/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD4

no data in this date range for  DB ,  10/02/22  to  10/03/22 ! :)


In [3]:
dataf

""
